In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
num_words = 10000
MAX_SEQUENCE_LENGTH = 16
WORD_EMBEDDING_DIM = 200
batch_size = 640
ENCODING_DIM = 768
NUM_DATAPOINT = -1 # All in the file
NUM_ICD_CODES = 2833
NUM_EPOCHS = 2
VALIDATION_SPLIT = 0.2
PATIENCE = 5
TOP_K_PER_SENTENCE = 10
similarity_threshold = 0.7

THRESHOLD = -1 # TODO

ICD_DESC_FILENAME = '/kaggle/input/mimicdata/ICD_desc_with_freq.csv'
#DD_FILENAME = 'MIMIC_III_Final_1000_clean_shorted.csv'
#DD_FILENAME = 'MIMIC-III-Final_5000_clean_shortened.csv'
DD_FILENAME = '/kaggle/input/mimic-clean-text/MIMIC-III-Final_cleaned.csv'
#EMB_FILE = '/kaggle/input/pmcmodel/PMC-w2v.bin'

#RESULT_FILENAME = 'Experiment Results.xlsx'

In [ ]:
def convert_to_unicode(text):
  """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text.decode("utf-8", "ignore")
    elif isinstance(text, unicode):
      return text
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")

In [ ]:
import re
import numpy as np
import six
import tensorflow as tf

In [ ]:
def cleanText(sentence):
        sentence = re.sub(
        r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(sentence))
        sentence = re.sub(r"[ ]+", " ", sentence)
        sentence = re.sub(r"\!+", "!", sentence)
        sentence = re.sub(r"\,+", ",", sentence)
        sentence = re.sub(r"\?+", "?", sentence)
        sentence = re.sub(r'[0-9]', " ", sentence)
        sentence = re.sub(r'[^\w\s]','',sentence)
        sentence = re.sub(r'(?:^| )\w(?:$| )', ' ', (sentence)).strip()#AKS
        sentence = sentence.lower()
        #return [tok.text for tok in self.nlp.tokenizer(sentence) if tok.text != " "]
        return sentence

In [ ]:
def processTextIntoSentences(text):
    if(not len(text)):
        print("\nGiven string is empty. Please check again")
    sentenceSize  = 125
    #sentenceSize = 256
    line = convert_to_unicode(text)
    line  = cleanText(text)
    tokens = tokenizer.tokenize(line)
    #print("\nTokens", tokens)
    if(tokens):
        all_tokens = [tokens[i: i+sentenceSize] for i in range(0, len(tokens), sentenceSize)]
        #print("\nbatched tokens", len(all_tokens), all_tokens)
        processedTokens = []
        for token_ in all_tokens:
            processedTokens.append(" ".join(token_))
        #print(len(processedTokens[1].split()))
        return ("|".join(processedTokens))

In [ ]:
import pickle
import tqdm
from collections import Counter

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import re
import unicodedata
import six
import tensorflow as tf

In [ ]:
data =  pd.read_csv('/kaggle/input/mimic-clean-text/MIMIC-III-Final_cleaned.csv')[:15000]

In [ ]:
class BasicTokenizer(object):
  """Runs basic tokenization (punctuation splitting, lower casing, etc.)."""

  def __init__(self, do_lower_case=True):
    """Constructs a BasicTokenizer.

    Args:
      do_lower_case: Whether to lower case the input.
    """
    self.do_lower_case = do_lower_case

  def tokenize(self, text):
    """Tokenizes a piece of text."""
    text = convert_to_unicode(text)
    text = self._clean_text(text)#perform invalid char removal and whitespace removal

    # This was added on November 1st, 2018 for the multilingual and Chinese
    # models. This is also applied to the English models now, but it doesn't
    # matter since the English models were not trained on any Chinese data
    # and generally don't have any Chinese data in them (there are Chinese
    # characters in the vocabulary because Wikipedia does have some Chinese
    # words in the English Wikipedia.).
    text = self._tokenize_chinese_chars(text)

    orig_tokens = whitespace_tokenize(text)
    split_tokens = []
    for token in orig_tokens:
      if self.do_lower_case:
        token = token.lower()
        token = self._run_strip_accents(token)
      split_tokens.extend(self._run_split_on_punc(token))

    output_tokens = whitespace_tokenize(" ".join(split_tokens))
    return output_tokens

  def _run_strip_accents(self, text):
    """Strips accents from a piece of text."""
    text = unicodedata.normalize("NFD", text)
    output = []
    for char in text:
      cat = unicodedata.category(char)
      if cat == "Mn":
        continue
      output.append(char)
    return "".join(output)

  def _run_split_on_punc(self, text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
      char = chars[i]
      if _is_punctuation(char):
        output.append([char])
        start_new_word = True
      else:
        if start_new_word:
          output.append([])
        start_new_word = False
        output[-1].append(char)
      i += 1

    return ["".join(x) for x in output]

  def _tokenize_chinese_chars(self, text):
    """Adds whitespace around any CJK character."""
    output = []
    for char in text:
      cp = ord(char)
      if self._is_chinese_char(cp):
        output.append(" ")
        output.append(char)
        output.append(" ")
      else:
        output.append(char)
    return "".join(output)

  def _is_chinese_char(self, cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
        (cp >= 0x3400 and cp <= 0x4DBF) or  #
        (cp >= 0x20000 and cp <= 0x2A6DF) or  #
        (cp >= 0x2A700 and cp <= 0x2B73F) or  #
        (cp >= 0x2B740 and cp <= 0x2B81F) or  #
        (cp >= 0x2B820 and cp <= 0x2CEAF) or
        (cp >= 0xF900 and cp <= 0xFAFF) or  #
        (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
      return True

    return False

  def _clean_text(self, text):
    """Performs invalid character removal and whitespace cleanup on text."""
    output = []
    for char in text:
      cp = ord(char)
      if cp == 0 or cp == 0xfffd or _is_control(char):
        continue
      if _is_whitespace(char):
        output.append(" ")
      else:
        output.append(char)
    return "".join(output)

In [ ]:
def load_vocab(vocab_file):
  """Loads a vocabulary file into a dictionary."""
  vocab = collections.OrderedDict()
  index = 0
  #with tf.gfile.GFile(vocab_file, "r") as reader:
  with tf.io.gfile.GFile(vocab_file, "r") as reader:
    while True:
      token = convert_to_unicode(reader.readline())
      if not token:
        break
      token = (token.strip()).split("\t")
      vocab[token[0]] = index
      index += 1
  print("\nVocab loading done!")
  return vocab

In [ ]:
class WordpieceTokenizer(object):
  """Runs WordPiece tokenziation."""

  def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=200):
    self.vocab = vocab
    self.unk_token = unk_token
    self.max_input_chars_per_word = max_input_chars_per_word

  def tokenize(self, text):
    """Tokenizes a piece of text into its word pieces.

    This uses a greedy longest-match-first algorithm to perform tokenization
    using the given vocabulary.

    For example:
      input = "unaffable"
      output = ["un", "##aff", "##able"]

    Args:
      text: A single token or whitespace separated tokens. This should have
        already been passed through `BasicTokenizer.

    Returns:
      A list of wordpiece tokens.
    """

    text = convert_to_unicode(text)

    output_tokens = []
    for token in whitespace_tokenize(text):
      chars = list(token)
      if len(chars) > self.max_input_chars_per_word:
        output_tokens.append(self.unk_token)
        #print("\nunknown token inserted")
        continue

      is_bad = False
      start = 0
      sub_tokens = []
      while start < len(chars):
        end = len(chars)
        cur_substr = None
        while start < end:
          substr = "".join(chars[start:end])
          if start > 0:
            substr = "##" + substr
          if substr in self.vocab:
            cur_substr = substr
            #print("\nknown token inserted", cur_substr)
            break
          end -= 1
        if cur_substr is None:
          is_bad = True
          break
        sub_tokens.append(cur_substr)
        start = end

      if is_bad:
        output_tokens.append(self.unk_token)
        #print("\nunknown token inserted") #This is where unknown tokens are being inserted
      else:
        output_tokens.extend(sub_tokens)
    return output_tokens


def _is_whitespace(char):
  """Checks whether `chars` is a whitespace character."""
  # \t, \n, and \r are technically contorl characters but we treat them
  # as whitespace since they are generally considered as such.
  if char == " " or char == "\t" or char == "\n" or char == "\r":
    return True
  cat = unicodedata.category(char)
  if cat == "Zs":
    return True
  return False


def _is_control(char):
  """Checks whether `chars` is a control character."""
  # These are technically control characters but we count them as whitespace
  # characters.
  if char == "\t" or char == "\n" or char == "\r":
    return False
  cat = unicodedata.category(char)
  if cat in ("Cc", "Cf"):
    return True
  return False


def _is_punctuation(char):
  """Checks whether `chars` is a punctuation character."""
  cp = ord(char)
  # We treat all non-letter/number ASCII as punctuation.
  # Characters such as "^", "$", and "`" are not in the Unicode
  # Punctuation class but we treat them as punctuation anyways, for
  # consistency.
  if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
      (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
    return True
  cat = unicodedata.category(char)
  if cat.startswith("P"):
    return True
  return False

In [ ]:
class FullTokenizer(object):
  """Runs end-to-end tokenziation."""

  def __init__(self, vocab_file, do_lower_case=True):
    self.vocab = load_vocab(vocab_file)#Create Vocab dist with index 0
    self.inv_vocab = {v: k for k, v in self.vocab.items()}#Get vocab dict with index as key and token as value
    self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case)#Initialize the basic tokenizer 
    self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      #for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(token)

    return split_tokens

  def convert_tokens_to_ids(self, tokens):
    return convert_by_vocab(self.vocab, tokens)

  def convert_ids_to_tokens(self, ids):
    return convert_by_vocab(self.inv_vocab, ids)

In [ ]:
def convert_by_vocab(vocab, items):
  """Converts a sequence of [tokens|ids] using the vocab."""
  output = []
  for item in items:
    output.append(vocab[item])
  return output


def convert_tokens_to_ids(vocab, tokens):
  return convert_by_vocab(vocab, tokens)


def convert_ids_to_tokens(inv_vocab, ids):
  return convert_by_vocab(inv_vocab, ids)


def whitespace_tokenize(text):
  """Runs basic whitespace cleaning and splitting on a piece of text."""
  text = text.strip()
  if not text:
    return []
  tokens = text.split()
  return tokens

In [ ]:
tokenizer = FullTokenizer(
      vocab_file='/kaggle/input/autoencoderdecoderonpmcembeddingvocab/Vocab.txt', do_lower_case=True)

In [ ]:
data['processedText'] = data.apply(lambda x:processTextIntoSentences(x['clean_text']) , axis=1)

In [ ]:
data.shape, data.columns

In [ ]:
texts = ''
trainingDocs = list(data['processedText'])
for doc in trainingDocs:
    texts = texts + doc


In [ ]:
len(texts.split())

In [ ]:
class TorchVocab(object):
    """Defines a vocabulary object that will be used to numericalize a field.
    Attributes:
        freqs: A collections.Counter object holding the frequencies of tokens
            in the data used to build the Vocab.
        stoi: A collections.defaultdict instance mapping token strings to
            numerical identifiers.
        itos: A list of token strings indexed by their numerical identifiers.
    """

    def __init__(self, counter, max_size=None, min_freq=1, specials=['<pad>', '<oov>'],
                 vectors=None, unk_init=None, vectors_cache=None):
        """Create a Vocab object from a collections.Counter.
        Arguments:
            counter: collections.Counter object holding the frequencies of
                each value found in the data.
            max_size: The maximum size of the vocabulary, or None for no
                maximum. Default: None.
            min_freq: The minimum frequency needed to include a token in the
                vocabulary. Values less than 1 will be set to 1. Default: 1.
            specials: The list of special tokens (e.g., padding or eos) that
                will be prepended to the vocabulary in addition to an <unk>
                token. Default: ['<pad>']
            vectors: One of either the available pretrained vectors
                or custom pretrained vectors (see Vocab.load_vectors);
                or a list of aforementioned vectors
            unk_init (callback): by default, initialize out-of-vocabulary word vectors
                to zero vectors; can be any function that takes in a Tensor and
                returns a Tensor of the same size. Default: torch.Tensor.zero_
            vectors_cache: directory for cached vectors. Default: '.vector_cache'
        """
        self.freqs = counter
        counter = counter.copy()
        min_freq = max(min_freq, 1)

        self.itos = list(specials)
        # frequencies of special tokens are not counted when building vocabulary
        # in frequency order
        for tok in specials:
            del counter[tok]

        max_size = None if max_size is None else max_size + len(self.itos)

        # sort by frequency, then alphabetically
        words_and_frequencies = sorted(counter.items(), key=lambda tup: tup[0])
        words_and_frequencies.sort(key=lambda tup: tup[1], reverse=True)

        for word, freq in words_and_frequencies:
            if freq < min_freq or len(self.itos) == max_size:
                break
            self.itos.append(word)

        # stoi is simply a reverse dict for itos
        self.stoi = {tok: i for i, tok in enumerate(self.itos)}

        self.vectors = None
        if vectors is not None:
            self.load_vectors(vectors, unk_init=unk_init, cache=vectors_cache)
        else:
            assert unk_init is None and vectors_cache is None

    def __eq__(self, other):
        if self.freqs != other.freqs:
            return False
        if self.stoi != other.stoi:
            return False
        if self.itos != other.itos:
            return False
        if self.vectors != other.vectors:
            return False
        return True

    def __len__(self):
        return len(self.itos)

    def vocab_rerank(self):
        self.stoi = {word: i for i, word in enumerate(self.itos)}

    def extend(self, v, sort=False):
        words = sorted(v.itos) if sort else v.itos
        for w in words:
            if w not in self.stoi:
                self.itos.append(w)
                self.stoi[w] = len(self.itos) - 1


class Vocab(TorchVocab):
    def __init__(self, counter, max_size=None, min_freq=1):
        self.pad_index = 0
        self.unk_index = 1
        self.eos_index = 2
        self.sos_index = 3
        self.mask_index = 4
        super().__init__(counter, specials=["<pad>", "<unk>", "<eos>", "<sos>", "<mask>"],
                         max_size=max_size, min_freq=min_freq)

    def to_seq(self, sentece, seq_len, with_eos=False, with_sos=False) -> list:
        pass

    def from_seq(self, seq, join=False, with_pad=False):
        pass

    @staticmethod
    def load_vocab(vocab_path: str) -> 'Vocab':
        with open(vocab_path, "rb") as f:
            return pickle.load(f)

    def save_vocab(self, vocab_path):
        with open(vocab_path, "wb") as f:
            pickle.dump(self, f)


# Building Vocab with text files
class WordVocab(Vocab):
    def __init__(self, texts, max_size=None, min_freq=1):
        print("Building Vocab")
        counter = Counter()
        count = 0
        words = ((texts.replace("\n", "").replace("\t", "")).replace("|"," ")).split()
        for word in words:
            if(not count % 10000):
                print(word)
            counter[word] += 1
            count += 1
        print("\nTotal line", count)
        super().__init__(counter, max_size=max_size, min_freq=min_freq)

    def to_seq(self, sentence, seq_len=None, with_eos=False, with_sos=False, with_len=False):
        if isinstance(sentence, str):
            sentence = sentence.split()

        seq = [self.stoi.get(word, self.unk_index) for word in sentence]

        if with_eos:
            seq += [self.eos_index]  # this would be index 1
        if with_sos:
            seq = [self.sos_index] + seq

        origin_seq_len = len(seq)

        if seq_len is None:
            pass
        elif len(seq) <= seq_len:
            seq += [self.pad_index for _ in range(seq_len - len(seq))]
        else:
            seq = seq[:seq_len]

        return (seq, origin_seq_len) if with_len else seq

    def from_seq(self, seq, join=False, with_pad=False):
        words = [self.itos[idx]
                 if idx < len(self.itos)
                 else "<%d>" % idx
                 for idx in seq
                 if not with_pad or idx != self.pad_index]

        return " ".join(words) if join else words

    @staticmethod
    def load_vocab(vocab_path: str) -> 'WordVocab':
        with open(vocab_path, "rb") as f:
            return pickle.load(f)

In [ ]:
newvocab = WordVocab(texts,100000,1)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import math

In [ ]:
class SegmentEmbedding(nn.Embedding):
    def __init__(self, embed_size=512):
        super().__init__(3, embed_size, padding_idx=0)

In [ ]:
class PositionalEmbedding(nn.Module):

    def __init__(self, d_model, max_len=256):
        super().__init__()

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]

In [ ]:
class TokenEmbedding(nn.Embedding):
    def __init__(self, vocab_size, embed_size=512):
        super().__init__(vocab_size, embed_size, padding_idx=0)

In [ ]:
class BERTEmbedding(nn.Module):
    """
    BERT Embedding which is consisted with under features
        1. TokenEmbedding : normal embedding matrix
        2. PositionalEmbedding : adding positional information using sin, cos
        2. SegmentEmbedding : adding sentence segment info, (sent_A:1, sent_B:2)

        sum of all these features are output of BERTEmbedding
    """

    def __init__(self, vocab_size, embed_size, dropout=0.1):
        """
        :param vocab_size: total vocab size
        :param embed_size: embedding size of token embedding
        :param dropout: dropout rate
        """
        super().__init__()
        self.token = TokenEmbedding(vocab_size=vocab_size, embed_size=embed_size)
        self.position = PositionalEmbedding(d_model=self.token.embedding_dim)
        self.segment = SegmentEmbedding(embed_size=self.token.embedding_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.embed_size = embed_size

    def forward(self, sequence, segment_label):
        x = self.token(sequence) + self.position(sequence) + self.segment(segment_label)
        return self.dropout(x)

In [ ]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = GELU()

    def forward(self, x):
        return self.w_2(self.dropout(self.activation(self.w_1(x))))


In [ ]:
class GELU(nn.Module):
    """
    Paper Section 3.4, last paragraph notice that BERT used the GELU instead of RELU
    """

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

In [ ]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))


In [ ]:
class Attention(nn.Module):
    """
    Compute 'Scaled Dot Product Attention
    """

    def forward(self, query, key, value, mask=None, dropout=None):
        scores = torch.matmul(query, key.transpose(-2, -1)) \
                 / math.sqrt(query.size(-1))

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        p_attn = F.softmax(scores, dim=-1)

        if dropout is not None:
            p_attn = dropout(p_attn)

        return torch.matmul(p_attn, value), p_attn

In [ ]:
class MultiHeadedAttention(nn.Module):
    """
    Take in model size and number of heads.
    """

    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0

        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h

        self.linear_layers = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(3)])
        self.output_linear = nn.Linear(d_model, d_model)
        self.attention = Attention()

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [l(x).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
                             for l, x in zip(self.linear_layers, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, attn = self.attention(query, key, value, mask=mask, dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.h * self.d_k)

        return self.output_linear(x)


In [ ]:
class TransformerBlock(nn.Module):
    """
    Bidirectional Encoder = Transformer (self-attention)
    Transformer = MultiHead_Attention + Feed_Forward with sublayer connection
    """

    def __init__(self, hidden, attn_heads, feed_forward_hidden, dropout):
        """
        :param hidden: hidden size of transformer
        :param attn_heads: head sizes of multi-head attention
        :param feed_forward_hidden: feed_forward_hidden, usually 4*hidden_size
        :param dropout: dropout rate
        """

        super().__init__()
        self.attention = MultiHeadedAttention(h=attn_heads, d_model=hidden)
        self.feed_forward = PositionwiseFeedForward(d_model=hidden, d_ff=feed_forward_hidden, dropout=dropout)
        self.input_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.output_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, mask):
        x = self.input_sublayer(x, lambda _x: self.attention.forward(_x, _x, _x, mask=mask))
        x = self.output_sublayer(x, self.feed_forward)
        return self.dropout(x)

In [ ]:
class BERT(nn.Module):
    """
    BERT model : Bidirectional Encoder Representations from Transformers.
    """

    def __init__(self, vocab_size, hidden=768, n_layers=12, attn_heads=12, dropout=0.1):
        """
        :param vocab_size: vocab_size of total words
        :param hidden: BERT model hidden size
        :param n_layers: numbers of Transformer blocks(layers)
        :param attn_heads: number of attention heads
        :param dropout: dropout rate
        """

        super().__init__()
        self.hidden = hidden
        self.n_layers = n_layers
        self.attn_heads = attn_heads

        # paper noted they used 4*hidden_size for ff_network_hidden_size
        self.feed_forward_hidden = hidden * 4

        # embedding for BERT, sum of positional, segment, token embeddings
        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=hidden)

        # multi-layers transformer blocks, deep network
        self.transformer_blocks = nn.ModuleList(
            [TransformerBlock(hidden, attn_heads, hidden * 4, dropout) for _ in range(n_layers)])

    def forward(self, x, segment_info):
        # attention masking for padded token
        # torch.ByteTensor([batch_size, 1, seq_len, seq_len)
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)

        # embedding the indexed sequence to sequence of vectors
        x = self.embedding(x, segment_info)

        # running over multiple transformer blocks
        for transformer in self.transformer_blocks:
            x = transformer.forward(x, mask)

        return x

In [ ]:
bert = BERT(len(newvocab), hidden=200, n_layers=8, attn_heads=5)

In [ ]:
class BERTLM(nn.Module):
    """
    BERT Language Model
    Next Sentence Prediction Model + Masked Language Model
    """

    def __init__(self, bert: BERT, vocab_size):
        """
        :param bert: BERT model which should be trained
        :param vocab_size: total vocab size for masked_lm
        """

        super().__init__()
        self.bert = bert
        self.next_sentence = NextSentencePrediction(self.bert.hidden)
        self.mask_lm = MaskedLanguageModel(self.bert.hidden, vocab_size)

    def forward(self, x, segment_label):
        x = self.bert(x, segment_label)
        return x#self.next_sentence(x), 


class NextSentencePrediction(nn.Module):
    """
    2-class classification model : is_next, is_not_next
    """

    def __init__(self, hidden):
        """
        :param hidden: BERT model output size
        """
        super().__init__()
        self.linear = nn.Linear(hidden, 2)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x[:, 0]))


class MaskedLanguageModel(nn.Module):
    """
    predicting origin token from masked input sequence
    n-class classification problem, n-class = vocab_size
    """

    def __init__(self, hidden, vocab_size):
        """
        :param hidden: output size of BERT model
        :param vocab_size: total vocab size
        """
        super().__init__()
        self.linear = nn.Linear(hidden, vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x))

In [ ]:
cuda_condition = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda_condition else "cpu")

In [ ]:
model = BERTLM(bert, len(newvocab)).to(device)

In [ ]:
#model = BERTLM(bert, 36841).to(device)

In [ ]:
model.load_state_dict(torch.load('/kaggle/input/bert-pretrain-bothtask/45_model_weights.pth'))
model.to(device)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
#list(model.named_parameters())

In [ ]:
print(model.transformer_blocks)

In [ ]:
for layer in model.transformer_blocks:
    layer.trainable = False

In [ ]:
import argparse

In [ ]:
mimicFilePath  =  '/kaggle/input/mimic-clean-text/MIMIC-III-Final_cleaned.csv'#os.path.join(dirname,'data\MIMIC-III-Final_cleaned.csv')
icdCodeFilepath  = '/kaggle/input/mimicdata/ICD_desc_with_freq.csv'#os.path.join(dirname,'data\ICD_desc_with_freq.csv')

In [ ]:
def parseArgs():
    parser = argparse.ArgumentParser()
    parser.add_argument('-mimicPath', required=False, default= 'mimicFilePath')
    parser.add_argument('-icdPath', required=False, default = 'icdCodeFilepath')
    parser.add_argument('-src_lang', required=False, default = 'en_core_web_sm')
    #parser.add_argument('-trg_lang', required=False, default = 'fr')
    parser.add_argument('-trainSize', required=False, default = 15000)
    parser.add_argument('-valSize', required=False, default = 500)
    parser.add_argument('-topCodes', required=False, default = 2833)
    parser.add_argument('-min_freq', required=False, default = 2)
    parser.add_argument('-no_cuda', action='store_true')
    parser.add_argument('-SGDR', action='store_false')
    parser.add_argument('-epochs', type=int, default=8)
    parser.add_argument('-d_model', type=int, default=512)
    parser.add_argument('-n_layers', type=int, default=6)
    parser.add_argument('-heads', type=int, default=8)
    parser.add_argument('-dropout', type=int, default=0.1)
    parser.add_argument('-batchsize', type=int, default=1)
    parser.add_argument('-printevery', type=int, default=100)
    parser.add_argument('-lr', type=int, default=0.0001)
    parser.add_argument('-load_weights')
    parser.add_argument('-create_valset', action='store_true')
    parser.add_argument('-max_strlen', type=int, default=80)
    parser.add_argument('-floyd', action='store_false')
    parser.add_argument('-checkpoint', type=int, default=1)
    opt = parser.parse_args("")

In [ ]:
opt = parseArgs()
opt

In [ ]:
mimicFilePath  =  '/kaggle/input/mimic-clean-text/MIMIC-III-Final_cleaned.csv'#os.path.join(dirname,'data\MIMIC-III-Final_cleaned.csv')
icdCodeFilepath  = '/kaggle/input/mimicdata/ICD_desc_with_freq.csv'#os.path.join(dirname,'data\ICD_desc_with_freq.csv')
#embedFilePath = '/kaggle/input/pmcmodel/PMC-w2v.bin' #os.path.join(dirname,'data\pmcModel\PMC-w2v.bin')
#vocabFilePath = '/kaggle/input/autoencoderdecoderonpmcembeddingvocab/Vocab.txt' #os.path.join(dirname, 'data\Vocab.txt')
#opt = parseArgs()

#opt.vocabFile  = vocabFilePath
#opt.embedFile = embedFilePath

In [ ]:
#opt.mimicPath 


In [ ]:
import pandas as pd
import spacy
#from Tokenize import tokenize
from collections import Counter
import torch
from torch.nn.utils.rnn import pad_sequence

def CreateCodeDescpMap(opt):
	#mimicData = pd.read_csv(opt.mimicPath)[:opt.trainSize]
	icdCodes = pd.read_csv(icdCodeFilepath)[:2833]
	icdCodeDescDict = {}
	codes  = list(icdCodes["Code"])
	Desc  = list(icdCodes["Long Description"])
	code2Index = {}
	for i , code in enumerate(codes):
		icdCodeDescDict[code] = Desc[i]
		code2Index[code] = i
	return icdCodeDescDict,code2Index

def CreateTextFileLableMap(opt):
	mimicData = pd.read_csv(mimicFilePath)[:150]
	icdCodeDescDict,code2Index = CreateCodeDescpMap(opt)
	trueLabels = list(mimicData['ICD9_CODE'])
	textFileLabel = {}
	for i , trueLabel in enumerate(trueLabels):
		textFileLabel[i+1]  = [y for y in trueLabel[2:-2].split("', '")if y in  icdCodeDescDict]
	return textFileLabel,icdCodeDescDict,code2Index


def processList(opt, tokensLists):
	vocabDict = {}
	VocabDictFinal  = {'<UNK>':1,'<PAD>':0}
	for i, tokensList in enumerate(tokensLists):
		if(not i%1000):
			print("\ndoc processed ",i)
		for token in tokensList:
			if token in vocabDict:
				count  = vocabDict[token]
				vocabDict[token] = count + 1
			else:
				vocabDict[token] = 1
	minimumTokenCountInVocab  = opt.min_freq
	for token, count in vocabDict.items():
		if(count>= minimumTokenCountInVocab and token not in VocabDictFinal):
			VocabDictFinal[token] = len(VocabDictFinal)
	return VocabDictFinal
	
	
	
def CreateVocab(opt,tokensLists):
	print(len(tokensLists))
	minimumTokenCountInVocab  = opt.min_freq
	allTokensLost = []
	for i, tokensList in enumerate(tokensLists):
		if(not i%1000):
			print("Processed documents",(i))
		allTokensLost = allTokensLost+tokensList
	tokenCountDict = Counter(allTokensLost)
	print(len(allTokensLost),len(tokenCountDict))
	VocabDict  = {'<UNK>':1,'<PAD>':2}
	#VocabDict = {VocabDict[token] = len(VocabDict)+1 for token , count in tokenCountDict.items() if count>=minimumTokenCountInVocab}
	for token, count in tokenCountDict.items():
		if(count>= minimumTokenCountInVocab and token not in VocabDict):
			VocabDict[token] = len(VocabDict)+1
	return VocabDict

def CreateVocabOnMimicTrainCodeDesc(opt, icdCodeDescDict):
	tok = tokenize(opt)
	mimicData = pd.read_csv(opt.mimicPath)[:opt.trainSize]
	mimicTexts = list(mimicData['clean_text'])
	mimicTokensList  = [tok.tokenizer(texts) for texts in mimicTexts ]
	icdCodesTokenList = [tok.tokenizer(codeTexts) for code,codeTexts in icdCodeDescDict.items()]
	print(len(mimicTokensList),len(icdCodesTokenList))
	#return (CreateVocab(opt,(mimicTokensList+icdCodesTokenList)))
	return (processList(opt,(mimicTokensList+icdCodesTokenList)))
	
def generateTensor(opt,tok, texts, vocabToIndexDict):
	textTokens  = tok.tokenizer(texts)
	textTokensToIndex  = [vocabToIndexDict[token] if token in vocabToIndexDict else vocabToIndexDict['<UNK>'] for token in textTokens ]
	return torch.tensor([textTokensToIndex])

def generateTensorForTrain(opt,tok, texts, vocabToIndexDict):
	textTokens  = tok.tokenizer(texts)
	textTokensToIndex  = [vocabToIndexDict[token] if token in vocabToIndexDict else vocabToIndexDict['<UNK>'] for token in textTokens ]
	print("\nchecking the shape of the train tensor",torch.tensor(textTokensToIndex).shape)
	return torch.tensor(textTokensToIndex)


def findMaxLength(mimicTextsTrainData):
	maxLen = 0
	for texts in mimicTextsTrainData:
		if(len(texts)>maxLen):
			maxlen = len(texts)
	return maxLen
def padTrainData(opt, mimicTextsTrainData,tok, vocabToIndexDict):
	trainTensor = []
	batchsize = opt.batchsize
	docIndex = 0
	batchDocs = []
	while(docIndex < opt.trainSize):
		batchDocs.append(generateTensorForTrain(opt,tok, mimicTextsTrainData[docIndex], vocabToIndexDict))
		if(len(batchDocs) == batchsize or (docIndex + 1== opt.trainSize)):
			trainTensor.append(torch.nn.utils.rnn.pad_sequence(batchDocs, batch_first=True))
			batchDocs = []
			docIndex += 1
		else:
			docIndex += 1
	return trainTensor

def processMimicTexts(opt,mimicTextsTrainData,tok,vocabToIndexDict):
    trainTensor = []
    print("\nTrain data size is",len(mimicTextsTrainData))
    for texts in mimicTextsTrainData:
        mimicTokens  = tok.tokenizer(texts)
        mimicTokens  = [vocabToIndexDict[token] if token in vocabToIndexDict else vocabToIndexDict['<UNK>'] for token in mimicTokens ]
        mimicTokens = [mimicTokens[i: i+512] for i in range(0, len(mimicTokens), 512)]
        finalList =[torch.tensor(i) for  i in mimicTokens]
        finalList = pad_sequence(finalList, batch_first=True, padding_value=0)
        trainTensor.append(finalList)
    print("\nTraining data prepared")
    return trainTensor
    

def create_dataset(opt, vocabToIndexDict, icdCodeDescDict):
	tok = tokenize(opt)
	mimicData = pd.read_csv(opt.mimicPath)[:opt.trainSize]
	mimicTextsTrainData= list((mimicData[:opt.trainSize])['clean_text'])
	mimicTextsValData= list((mimicData[:opt.valSize])['clean_text'])
	icdCodes = pd.read_csv(opt.icdPath)[:opt.topCodes]
	Desc  = list(icdCodes["Long Description"])
	#maxLen = findMaxLength(mimicTextsTrainData)
	#trainOnMimicData = [generateTensor(opt,tok,texts,vocabToIndexDict) for texts in mimicTextsTrainData]
	#trainOnMimicData = padTrainData(opt,mimicTextsTrainData,tok,vocabToIndexDict)
	trainOnMimicData = processMimicTexts(opt,mimicTextsTrainData,tok,vocabToIndexDict)
	valOnMimicData = [generateTensor(opt,tok,texts,vocabToIndexDict) for texts in mimicTextsValData]
	trainOncodeDesc = [generateTensor(opt,tok,texts,vocabToIndexDict) for texts in Desc]
	return trainOnMimicData,valOnMimicData,trainOncodeDesc

In [ ]:
textFileLabel,icdCodeDescDict, code2Index = CreateTextFileLableMap(opt)
#opt.vocabToIndexDict, opt.indexToVocabDict = createVocab(opt,icdCodeDescDict)
#print(len(opt.indexToVocabDict))

In [ ]:
len(newvocab.stoi)

In [ ]:
processTextIntoSentences(data['processedText'][0])

In [ ]:
#model.eval()

In [ ]:
te = data['processedText'][0]

In [ ]:
from torch.utils.data import Dataset
import tqdm
import torch
import random

In [ ]:
class BERTDataset(Dataset):
    def __init__(self,vocab, seq_len, list_IDs, labels, encoding="utf-8", on_memory=True):
        self.vocab = vocab
        self.seq_len = seq_len

        self.on_memory = on_memory
        #self.corpus_lines = corpus_lines
        self.list_IDs  = list_IDs 
        self.encoding = encoding
        self.labels = labels
        self.texts  = data['processedText']


    def __len__(self):
        return len(self.list_IDs)

    def __getitem__(self, item):# if item count start from 1 and till the size of docuemts length 
        text_ = self.texts[item-1]
        text_ = text_.split('|')
        if(len(text_) % 2):
            text_.append(text_[-1])
        text_ = [text_[i: i+2] for i in range(0, len(text_), 2)]
        #print(len(text_))
        bert_input_ = self.generateTensor(text_)
        return bert_input_
        #return {key: torch.tensor(value) for key, value in output.items()}
    def generateTensor(self, text):
        numberOfLines  = len(text)
        bert_input_ = []
        bert_label_ = []
        segment_label_ = []
        is_next_ = []
        for i in range(numberOfLines):
            t1, t2, is_next_label = self.random_sent(i, text)#generate the label for next sentence prediction in the form of 0 or 1
            t1_random, t1_label = self.random_word(t1)#This will generate the masked token for sentence A and the label index for the masked token 
            t2_random, t2_label = self.random_word(t2)

        # [CLS] tag = SOS tag, [SEP] tag = EOS tag
            t1 = [self.vocab.sos_index] + t1_random + [self.vocab.eos_index]
            t2 = t2_random + [self.vocab.eos_index]

            t1_label = [self.vocab.pad_index] + t1_label + [self.vocab.pad_index]
            t2_label = t2_label + [self.vocab.pad_index]

            segment_label = ([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:self.seq_len]
            bert_input = (t1 + t2)[:self.seq_len]
            bert_label = (t1_label + t2_label)[:self.seq_len]

            padding = [self.vocab.pad_index for _ in range(self.seq_len - len(bert_input))]#This would pad the input, output lable and sengement label up to the maximum length
            bert_input.extend(padding), bert_label.extend(padding), segment_label.extend(padding)
            bert_input_.append(bert_input)
            bert_label_.append(bert_label)
            segment_label_.append(segment_label)
            is_next_.append(is_next_label)
        return ([torch.tensor(bert_input_),torch.tensor(bert_label_),torch.tensor(segment_label_), torch.tensor(is_next_)])
    def random_word(self, sentence):
        tokens = sentence.split()
        output_label = []

        for i, token in enumerate(tokens):
            prob = random.random()
            if prob < 0.15:
                prob /= 0.15

                # 80% randomly change token to mask token
                if prob < 0.8:
                    tokens[i] = self.vocab.mask_index

                # 10% randomly change token to random token
                elif prob < 0.9:
                    tokens[i] = random.randrange(len(self.vocab))

                # 10% randomly change token to current token
                else:
                    tokens[i] = self.vocab.stoi.get(token, self.vocab.unk_index)

                output_label.append(self.vocab.stoi.get(token, self.vocab.unk_index))

            else:
                tokens[i] = self.vocab.stoi.get(token, self.vocab.unk_index)
                output_label.append(0)

        return tokens, output_label

    def random_sent(self, index,text):
        t1, t2 = self.get_corpus_line(index,text)

        # output_text, label(isNotNext:0, isNext:1)
        if random.random() > 0.5:
            return t1, t2, 1
        else:
            return t1, self.get_random_line(text), 0

    def get_corpus_line(self, item,text):
        if self.on_memory:
            return text[item][0], text[item][1]
        """else:
            line = self.file.__next__()
            if line is None:
                self.file.close()
                self.file = open(self.corpus_path, "r", encoding=self.encoding)
                line = self.file.__next__()

            t1, t2 = line[:-1].split("\t")
            return t1, t2"""

    def get_random_line(self,text):
        if self.on_memory:
            return text[random.randrange(len(text))][1]

In [ ]:
listIds = [i for i in range(data.shape[0])]

In [ ]:
#textFileLabel

In [ ]:
testDataSet = BERTDataset(newvocab,256, listIds, textFileLabel)

In [ ]:
testDataSet[1][0].shape

In [ ]:
trainTensors = testDataSet[1]

In [ ]:
#a, b, c = model(trainTensors[0].to(device), trainTensors[2].to(device))

In [ ]:
#c.shape

In [ ]:
newvocab.stoi.get('<unk>',newvocab.unk_index )

In [ ]:
def generateTensorsOnTextData(text):
    #text = text.split()
    sentenceSize  = 125
    line = convert_to_unicode(text)
    line  = cleanText(text)
    tokens = tokenizer.tokenize(line)
    #print("\ntotal token count ", len(tokens))
    padTokens  = []
    all_tokens = [tokens[i: i+sentenceSize] for i in range(0, len(tokens), sentenceSize)]
    if(len(all_tokens) == 1):
        padTokens.append(" ".join(all_tokens[0]))
        padTokens.append(" ".join(["<pad>" for _ in range(sentenceSize)]))
        #return padTokens
    else:
        if(not (len(all_tokens)% 2)):
            padTokens = [" ".join(tok) for tok in all_tokens]
            #return padTokens
        else:
            padTokens = [" ".join(tok) for tok in all_tokens]
            padTokens.append(" ".join(["<pad>" for _ in range(sentenceSize)]))
            #return padTokens
    text = [padTokens[i: i+2] for i in range(0, len(padTokens), 2)]
    bert_input_ = []
    bert_label_ = []
    segment_label_ = []
    is_next_ = []
    for i in range(len(text)):
            t1, t2, is_next_label = text[i][0], text[i][1], 1#(i, text)#generate the label for next sentence prediction in the form of 0 or 1
            t1 = [newvocab.stoi.get(tko, newvocab.unk_index) for tko in t1.split()]
            t2 = [newvocab.stoi.get(tko, newvocab.unk_index) for tko in t2.split()]

        # [CLS] tag = SOS tag, [SEP] tag = EOS tag
            t1 = [newvocab.sos_index] + t1 + [newvocab.eos_index]
            t2 = t2 + [newvocab.eos_index]
            segment_label = ([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:256]
            bert_input = (t1 + t2)[:256]
            padding = [newvocab.pad_index for _ in range(256 - len(bert_input))]#This would pad the input, output lable and sengement label up to the maximum length
            bert_input.extend(padding), segment_label.extend(padding)
            bert_input_.append(bert_input)
            segment_label_.append(segment_label)
    return torch.tensor(bert_input_), torch.tensor(segment_label_)

In [ ]:
#generateTensorsOnTextData(te)

In [ ]:
newvocab.stoi.get('<pad>', newvocab.unk_index)

In [ ]:
def get_word_embedding(text):
    a, b = generateTensorsOnTextData(text)
    embeddings = []
    #tensors = testDataSet[docNumber]
    c = model(a.to(device), b.to(device))
    c = c[:,1:,:]
    #print(c.shape)
    return c.contiguous().view(-1,200)
    

In [ ]:
get_word_embedding("this is the string").shape

In [ ]:
def get_sentence_embedding(sent):
    #words = tokenizer.tokenize(sent)
    #m = len(words)
    total_embedding = 0
    
    #Get wordwise Embeddings as tensors
    #sent_embedding = np.array([get_word_embedding(word, embedding) for word in words])
    sent_embedding = sent#.cpu().detach().numpy()#np.array([word.detach().to('cpu').numpy() for word in get_word_embedding(sent)])
    
    #Get the embedding of sentence by adding embedding tensors
    total_embedding = np.sum(sent_embedding, axis = 0)
    
    try:
        return total_embedding/np.linalg.norm(total_embedding, axis = 0, keepdims=True)
    except:
        #print('sent : ', sent)
        #print('m: ', m)
        return np.array(total_embedding/np.linalg.norm(total_embedding, axis = 0, keepdims=True))

In [ ]:
#get_sentence_embedding(get_word_embedding(1)).shape

In [ ]:
ICD_desc_filename = ICD_DESC_FILENAME
df_icd = pd.read_csv(ICD_desc_filename, encoding = 'latin1')

df_icd = df_icd[['Code', 'Long Description']]

print(df_icd.shape)

print('\n', df_icd.head())

In [ ]:
!pip install transformers==3.0.0
import os 
import sys
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import argparse
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_sequence
import time
import random
import math
import pickle
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.models import Model, load_model
from keras.layers import LSTM, Dense, TimeDistributed, Input, Masking, RepeatVector, Bidirectional, Embedding
nltk.download('punkt')
import string
punc = list(string.punctuation)
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english'))
print(stop_words)
from sklearn.metrics import pairwise
from sklearn.preprocessing import normalize
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import pickle
import csv
import os
from scipy.spatial.distance import cosine
import time
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.feature_extraction.text import CountVectorizer
import torch

In [ ]:
df_icd_cleaned = df_icd

#Delete df_icd
del df_icd

def clean_icd(x):
    sent = x
    sent_tokenized = word_tokenize(sent)
    
    clean_sent = ''
    
    for token in sent_tokenized:
        if token.lower() not in punc and token.lower() not in stop_words:
            clean_sent = clean_sent + ' ' + token.lower()
    
    x = clean_sent
    
    del clean_sent
    
    return x


df_icd_cleaned['Long Description'] = df_icd_cleaned.apply(lambda x: clean_icd(x['Long Description']), axis = 1)

df_icd_cleaned.head()

In [ ]:
icd_sentences = list(df_icd_cleaned.head(NUM_ICD_CODES)['Long Description']) # Number of sentences
icd_codes = list(df_icd_cleaned.head(NUM_ICD_CODES)['Code'])
icd_index = list(range(NUM_ICD_CODES))

print(len(icd_sentences), len(icd_codes), len(icd_index))
print("ICD_Sentences = ", len(icd_sentences))

In [ ]:
# 3 Dictionary of ICD index, ICD Code and ICD desc

# index, codes
dict_index_code = dict(zip(icd_index, icd_codes))

# codes, index
dict_code_index = dict(zip(icd_codes, icd_index))

#index, code desc
dict_index_desc = dict(zip(icd_index, icd_sentences))

#icd sentence embedding matrix slice
#icd_sent_emb_matrix = icd_sent_emb_matrix[:NUM_ICD_CODES]

In [ ]:
dict_index_code[500]

In [ ]:
DD_filename = DD_FILENAME
df_DD = pd.read_csv(DD_filename)
df_DD = df_DD[0:50]
df_DD.reset_index(drop=True, inplace= True)
FILESIZE = df_DD.shape[0]
df_DD.head()

In [ ]:
import spacy
from spacy.language import Language

@Language.component("custom_boundary")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == '...' or token.text == '.' or token.text =='!' :
            doc[token.i+1].is_sent_start = True
    return doc

custom_nlp = spacy.load('en_core_web_sm')

custom_nlp.add_pipe("custom_boundary", before='parser')

def removingNegativeSentence(text):
    cleanText = []
    negative_token = ["not","absence","no"]
    custom_doc = custom_nlp(text)
    custom_sentences = list(custom_doc.sents)
    for sentence in custom_sentences:
        #print (sentence.text)
        flag  = False
        for token in sentence:
            #print (type(token))
            if token.text in negative_token:
                #Print the Dependant Word to Neg words
                #print (token.text, token.tag_, token.head.text, token.dep_)
                #displacy.serve(sentence, style='dep')
                flag  = True
                break
        if(not flag):
            #print(sentence.text)
            cleanText.append(sentence.text)
    return (' '.join(cleanText))

In [ ]:
DD_Data = df_DD[['ICD9_CODE', 'clean_text']]
del df_DD
DD_Data

In [ ]:
#Negation Removal
DD_Data['clean_text'] = DD_Data['clean_text'].apply(lambda x: processTextIntoSentences(removingNegativeSentence(x)))
DD_Data.head()

In [ ]:
def get_tf_modified(docs):
    freqDict = {}
    for doc in docs:
        tokens  = tokenizer.tokenize(doc)
        for token in tokens:
            if(token in freqDict):
                freqDict[token] = int(freqDict[token]) + 1
            else:
                freqDict[token] = 1
    freq_vec = pd.DataFrame(freqDict.values(), index=freqDict.keys(), columns=['Frequency'])
    return  freq_vec, freqDict

In [ ]:
def get_tf(docs):
    
    vectorizer = CountVectorizer()#CountVectorizer(stop_words = 'english')
    X = vectorizer.fit_transform(docs)#(docs['sentence'])

    # place tf values in a pandas data frame
    tf_vecs = pd.DataFrame(X.T.todense(), index = vectorizer.get_feature_names())
    
    #Take sum accross sentences
    freq_vec = tf_vecs.sum(axis = 1)
    freq_vec = pd.DataFrame(freq_vec, index=freq_vec.index, columns=['Frequency'])
    
    return vectorizer, freq_vec

In [ ]:
def remove_unwanted_words(words, freq_vec):
    
    pruned_words = [x for x in words if x in freq_vec.index]
    
    #for i in words:
    #    if i in freq_vec.index:
    #        pruned_words.append(i)
    
    return pruned_words
    #return list(set(pruned_words))

In [ ]:
#tokenizer.tokenize(mimicTexts)
def get_tf_vector(sent, freq_vec):
    #tokenize sentence
    words = list(set(tokenizer.tokenize(sent)))
    
    #remove terms not occuring in freq_vec
    pruned_words = remove_unwanted_words(words, freq_vec)
    
    
    #get the slice of freq_vec using the remaining terms
    #print(words, pruned_words)
    freq_slice = freq_vec.loc[pruned_words, :]
    
    #L1 normalize to obtain tf vector
    tf_vec = freq_slice/np.linalg.norm(freq_slice, ord = 1)    

    return tf_vec, pruned_words

In [ ]:
def get_tf_vector_modified(sent, freq_vec):
    #tokenize sentence
    words = (tokenizer.tokenize(sent))
    
    #remove terms not occuring in freq_vec
    pruned_words = remove_unwanted_words(words, freq_vec)
    
    
    #get the slice of freq_vec using the remaining terms
    #print(words, pruned_words)
    freq_slice = freq_vec.loc[pruned_words, :]
    
    #L1 normalize to obtain tf vector
    tf_vec = freq_slice/np.linalg.norm(freq_slice, ord = 1)    

    return tf_vec, pruned_words

In [ ]:
def cosine_sim_matrix(s1, s2):
    
    #Get token-wise embedding
    #s1_token_emb = np.array([get_word_embedding(word, word_model) for word in s1])
    #s1_token_emb = np.array([word.to('cpu') for word in get_word_embedding(s1)])
    #s1_token_emb = np.array([word.detach().to('cpu').numpy() for word in get_word_embedding(s1)])
    #print(type(s1_token_emb))#x.detach().cpu().numpy()
    #Normalize
    s1_token_emb = s1
    s1_token_emb/=np.linalg.norm(s1_token_emb, axis = 1, ord = 2, keepdims=True)
    
    #Get token-wise embedding of icd text
    #s2_token_emb = np.array([get_word_embedding(word, word_model) for word in s2])
    #s2_token_emb = np.array([word.to('cpu') for word in get_word_embedding(s2)])
    #s2_token_emb = np.array([word.detach().to('cpu').numpy()for word in get_word_embedding(s2)])
    #Normalize
    #print(type(s2_token_emb))
    s2_token_emb = s2
    s2_token_emb/=np.linalg.norm(s2_token_emb, axis = 1, ord = 2, keepdims=True)
    
    
    sim_matrix = s1_token_emb.dot(s2_token_emb.T)
    #print("Here is the sim matrix ",(sim_matrix))
    
    return np.clip(sim_matrix, -1.0, 1.0) #Clip the Elements of the matrix

In [ ]:
freq_text_vector, freq_test = get_tf_modified(df_icd_cleaned['Long Description'])

In [ ]:
#freq_test

In [ ]:
(get_tf_vector("acute kidney failure acute ", freq_text_vector))

In [ ]:
(get_tf_vector_modified("acute kidney failure acute ", freq_text_vector))

In [ ]:
icd_sentences_embed_test = [[word.detach().to('cpu').numpy()for word in get_word_embedding(icd_text)] for icd_text in icd_sentences]

In [ ]:
#dict(freq_test)['Frequency']

In [ ]:
#(freq_test.loc['unspecified', :]).values

In [ ]:
#((icd_sentences_embed_test)[0])[0]

In [ ]:
#Reset Debugging Counter
k = 0
#Calculate the Similarity Score between 2 senteces
def RWMS_similarity_old(sent_1, sent_2, freq_vec,icd_text_embed):
    global k
    try:
        
        #get tf vectors
        #start = time.time()
        sent1_tf_vec, sent1_list = get_tf_vector(sent_1, freq_vec)
        #sent2_tf_vec, sent2 = get_tf_vector(sent_2, freq_vec)
        
        #tokenized sentences
        #sent1 = sent1#list(set(word_tokenize(sent_1)))
       # sent2 = list(set(word_tokenize(sent_2)))

        #Compute Cosine Similarity Matrix
        sim_mat = cosine_sim_matrix(icd_text_embed, sent_2)
        #print('Sim_mat:\n', sim_mat)
        
        #Initialise Similarity values:
        W_s_t = 0
        #W_t_s = 0

        #for S->t (icd_text -> dd text)
        max_cosine_vec = sim_mat.max(axis = 1)
        #print('Sim_mat:\n', sim_mat)
        
        # Take Elementwise product of max cos similarity vector and tf vector
        sim_vec = sent1_tf_vec.to_numpy().transpose() * max_cosine_vec
        
        #Take sum of all elements of sim_vec to obtain final similarity
        W_s_t = np.sum(sim_vec)
        
        del sim_mat, sent1_tf_vec#sent1, sent2
        
        #for t->S
        #for j in range(len(sent2)):
        #    w2 = sent2[j]
        #    sim = 0
        #    if w2 in density_tfidf.index:
        #        #index of word with max cosine similarity from sent1
        #        max_j = np.argmax(sim_mat.transpose()[j])

        #        #multiply the tfidf value of current word of sent2 with cosine similarity of the most similar from sent1
        #        sim = sim_mat.transpose()[j][max_j]*density_tfidf['s2_tfidf'][w2]
        #    else:
        #        sim = 0
        #    W_t_s+=sim
        
        
        #k = k + 1
        #print('k: ', k)

        return W_s_t#, sim_mat, max_cosine_vec, sent1_tf_vec, sent1, sent2   #min(W_s_t, W_t_s)
    except:
        #print('sent_1: ', sent_1)
        sent_1_emb = get_sentence_embedding(icd_text_embed)
        #print('sent_2: ', sent_2)
        sent_2_emb = get_sentence_embedding(sent_2)
        
        #k = k + 1
        #print('k: ', k)
        
        return sent_1_emb.dot(sent_2_emb.T)#, 0, 0, 0, 0, 0#cosine_sim(sent_1_emb, sent_2_emb)

In [ ]:
#Reset Debugging Counter
k = 0
#Calculate the Similarity Score between 2 senteces
def RWMS_similarity(sent_1, sent_2, freq_vec,icd_text_embed):
    global k
    try:
        
        #get tf vectors
        #start = time.time()
        sent1_tf_vec, sent1_list = get_tf_vector_modified(sent_1, freq_vec)
        #sent2_tf_vec, sent2 = get_tf_vector(sent_2, freq_vec)
        
        #tokenized sentences
        #sent1 = sent1#list(set(word_tokenize(sent_1)))
       # sent2 = list(set(word_tokenize(sent_2)))

        #Compute Cosine Similarity Matrix
        sim_mat = cosine_sim_matrix(icd_text_embed[:len(sent1_list)], sent_2)
        #print('Sim_mat:\n', sim_mat)
        
        #Initialise Similarity values:
        W_s_t = 0
        #W_t_s = 0

        #for S->t (icd_text -> dd text)
        max_cosine_vec = sim_mat.max(axis = 1)
        #print('Sim_mat:\n', sim_mat)
        
        # Take Elementwise product of max cos similarity vector and tf vector
        sim_vec = sent1_tf_vec.to_numpy().transpose() * max_cosine_vec
        
        #Take sum of all elements of sim_vec to obtain final similarity
        W_s_t = np.sum(sim_vec)
        
        del sim_mat, sent1_tf_vec#sent1, sent2
        
        #for t->S
        #for j in range(len(sent2)):
        #    w2 = sent2[j]
        #    sim = 0
        #    if w2 in density_tfidf.index:
        #        #index of word with max cosine similarity from sent1
        #        max_j = np.argmax(sim_mat.transpose()[j])

        #        #multiply the tfidf value of current word of sent2 with cosine similarity of the most similar from sent1
        #        sim = sim_mat.transpose()[j][max_j]*density_tfidf['s2_tfidf'][w2]
        #    else:
        #        sim = 0
        #    W_t_s+=sim
        
        
        #k = k + 1
        #print('k: ', k)

        return W_s_t#, sim_mat, max_cosine_vec, sent1_tf_vec, sent1, sent2   #min(W_s_t, W_t_s)
    except:
        #print('sent_1: ', sent_1)
        sent_1_emb = get_sentence_embedding(icd_text_embed)
        #print('sent_2: ', sent_2)
        sent_2_emb = get_sentence_embedding(sent_2)
        
        #k = k + 1
        #print('k: ', k)
        
        return sent_1_emb.dot(sent_2_emb.T)#, 0, 0, 0, 0, 0#cosine_sim(sent_1_emb, sent_2_emb)

In [ ]:
#Convert ICD Indices to hot vectors of length: NUM_ICD_CODE
def generate_binary(icd_index_list, total_icd = NUM_ICD_CODES):
    binary_icd = np.zeros(total_icd)
    for x in icd_index_list:
        binary_icd[x] = 1
        
    return binary_icd

In [ ]:
#Calculate Specificity

def calc_specificity(pred_vec, act_vec):
    m = len(pred_vec)
    
    specificity = []
    
    for i in range(m):
        pred, act = pred_vec[i], act_vec[i]
        dict_XY = {'pred':pred, 'act':act}
        res = pd.DataFrame(dict_XY, columns=['pred', 'act'])
        res['Sum'] = res['pred'] + res['act']
        
        pred_p = res[res.pred == 1]
        pred_n = res[res.act == 1]
        
        tp = len(res[res.Sum == 2])
        tn = len(res[res.Sum == 0]) 
        
        fp = len(pred_p[pred_p.act == 0])
        fn = len(pred_n[pred_n.act == 1])
        
        #Calculate specificity for this example:
        spec = tn/(tn + fp)
        
        specificity.append(spec)
    
    return sum(specificity) / float(m)

In [ ]:
def generateTensorsOnCodeTextData(text):
    #text = text.split()
    sentenceSize  = 125
    line = convert_to_unicode(text)
    line  = cleanText(text)
    tokens = tokenizer.tokenize(line)
    sentLength  = len(tokens)
    #print("\ntotal token count ", len(tokens))
    padTokens  = []
    all_tokens = [tokens[i: i+sentenceSize] for i in range(0, len(tokens), sentenceSize)]
    if(len(all_tokens) == 1):
        padTokens.append(" ".join(all_tokens[0]))
        padTokens.append(" ".join(["<pad>" for _ in range(sentenceSize)]))
        #return padTokens
    else:
        if(not (len(all_tokens)% 2)):
            padTokens = [" ".join(tok) for tok in all_tokens]
            #return padTokens
        else:
            padTokens = [" ".join(tok) for tok in all_tokens]
            padTokens.append(" ".join(["<pad>" for _ in range(sentenceSize)]))
            #return padTokens
    text = [padTokens[i: i+2] for i in range(0, len(padTokens), 2)]
    bert_input_ = []
    bert_label_ = []
    segment_label_ = []
    is_next_ = []
    for i in range(len(text)):
            t1, t2, is_next_label = text[i][0], text[i][1], 1#(i, text)#generate the label for next sentence prediction in the form of 0 or 1
            t1 = [newvocab.stoi.get(tko, newvocab.unk_index) for tko in t1.split()]
            t2 = [newvocab.stoi.get(tko, newvocab.unk_index) for tko in t2.split()]

        # [CLS] tag = SOS tag, [SEP] tag = EOS tag
            t1 = [newvocab.sos_index] + t1 + [newvocab.eos_index]
            t2 = t2 + [newvocab.eos_index]
            segment_label = ([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:256]
            bert_input = (t1 + t2)[:256]
            padding = [newvocab.pad_index for _ in range(256 - len(bert_input))]#This would pad the input, output lable and sengement label up to the maximum length
            bert_input.extend(padding), segment_label.extend(padding)
            bert_input_.append(bert_input)
            segment_label_.append(segment_label)
    return torch.tensor(bert_input_), torch.tensor(segment_label_)

In [ ]:
#Predict ICD 9 Codes

#construct the Term Frequency Vector using just the icd corpus
freq_vec, freq_vec_dict = get_tf_modified(df_icd_cleaned['Long Description'])

start_time = time.time()

#Calculate the Similarity of each document with each icd text
X = DD_Data['clean_text']#[0]
Y = list(DD_Data['ICD9_CODE'])
m = len(Y)

sim = []
icd_sentences_embed = [[word.detach().to('cpu').numpy()for word in get_word_embedding(icd_text)] for icd_text in icd_sentences]
print("\nEmbeddings saved for code")

for i in range(m):
    print("processing doc ",i)
    text = X[i]
    text_embed = [word.detach().to('cpu').numpy()for word in get_word_embedding(text)]
    #print(type(text_embed), text_embed)
    #Modified#Calculate cosine similarity of sentence with icd sentences
    sim.append(np.array([RWMS_similarity(icd_sentences[j], text_embed, freq_vec,icd_text_embed) for j, icd_text_embed in enumerate(icd_sentences_embed)]))#cosine_sim1_matrix(sent_emb)
    
    #reset debugging counter
    #global k
    #k = 0
    
    if (i+1)%10 == 0:
        print(str(i+1) + '/' + str(m) + ' records done.')
    
    if (i+1)%100 == 0:
        print('\nTime elapsed: ', (time.time() - start_time)/60.0, ' minutes')

    if (i+1)%1000 == 0:
        sim_df = pd.DataFrame(sim, dtype = float)
        sim_df.to_csv('Similarity_Data/Optimized_NP_RWMS_Negation_similarity_data_' + str(i+1) + '_records.csv')
        #sim_df.to_csv('Optimized_NP_RWMS_similarity_data_' + (i+1) + '_records.csv')

time_elapsed = time.time() - start_time
print('\nTime elapsed: ', time_elapsed/60.0, ' minutes')

#Save the similarity scores in an excel file
print('Saving Similarity scores in file......')
sim_df = pd.DataFrame(sim, dtype = float)
sim_df.to_csv('Similarity_Data_Optimized_NP_RWMS_Negation_similarity_data_' + str(m) + '_records.csv')
#sim_df.to_csv('Optimized_NP_RWMS_similarity_data_' + str(m) + '_records.csv')
print('Similarity Scores Saved!')

In [ ]:
#Load Similarity Data
sim = pd.read_csv('/kaggle/working/Similarity_Data_Optimized_NP_RWMS_Negation_similarity_data_'+str(m)+'_records.csv')
#sim = pd.read_csv('Optimized_NP_RWMS_similarity_data_' + str(m) + '_records.csv')

#Drop first column
sim = sim.drop(['Unnamed: 0'], axis = 1)

#Convert DataFrame to List
sim = np.array(sim.values.tolist())

In [ ]:
freq_vec

In [ ]:
#Save Predictions and Corresponding Scores per threshold

#Initialise Similarity Result Dataframe
results_sim = pd.DataFrame(columns = ['Actual Code', 
                                  'Predicted Codes',
                                  'Similarity Codes'])


threshold_list = [0.3,0.4,0.5,0.6, 0.7, 0.8, 0.9]

#Initialize Excel
#writer = pd.ExcelWriter('Similarity_Data/Code and Similarity Data - Unsupervised ICD - ' + str(m) + ' Records.xlsx', engine='xlsxwriter')

for thresh in threshold_list:

    #Print Threshold:
    print('\nCommenced for Similarity Threshold:', thresh)

    #X = DD_Data['clean_text'][0]
    #Y = list(DD_Data['ICD9_CODE'])

    data_input_format = []

    i = 0
    for i in range(m):
        #Convert Actual Codes to indices. Consider only those codes present in icd_codes
        act_codes = [str(y) for y in Y[i][2:-2].split("', '") if y in icd_codes]#To get rid of '' and ,
        act_codes_string = ', '.join(act_codes)

        #Apply thresholding to obtain indices of similar embeddings
        #print(sim)
        threshold_filter = sim[i] > thresh


        #Similarity >= threshold --> similarity 
        #Similarity < threshold --> 0
        sim_post_threshold = sim[i]*threshold_filter

        #Add indices of non zero elements to pred
        pred_indices = np.nonzero(sim_post_threshold)[0].tolist()

        #Get codes from indices
        pred_codes = [str(dict_index_code[p]) for p in pred_indices]
        #Convert to Comma Separated String
        pred_codes_string = ', '.join(pred_codes)
        #print([pred_codes_string])

        #Corresponding Similarity Scores of List of Indices
        sim_scores = [str(sim[i][j]) for j in pred_indices]
        #Convert to Comma Separated String
        sim_scores_string = ', '.join(sim_scores)
        #print([sim_scores_string])


        data_input = [act_codes_string, pred_codes_string, sim_scores_string]

        #Append
        data_input_format.append(data_input)

    df = pd.DataFrame(data_input_format, columns = ['Actual Codes', 'Predicted Codes', 'Similarity Scores'])
    df.to_csv('Similarity_Data_Negation - Code and Similarity Data - Unsupervised ICD - ' + str(m) + ' Records_' + str(thresh) + '_threshold.csv')
    #df.to_csv('NP - Predicted Codes and Similarity Data - Unsupervised ICD - ' + str(m) + ' Records_' + str(thresh) + '_threshold.csv')

In [ ]:
!pip install openpyxl
import openpyxl

In [ ]:
#Calculate Metrics

#Initialize Results Dataframe which will store the relevant metrics for each threshold
results = pd.DataFrame(columns = ['Number of Observations', 
                                  'Number of ICD Codes',
                                  'Threshold',
                                  'Precision', 
                                  'Recall', 
                                  'F1-Score', 
                                  'Specificity'])

#Compute Accuracy metrics based on thresholding        
#threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#threshold_list = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
threshold_list = [0.3,0.4,0.45,0.5 ,0.52,0.55,0.6, 0.7, 0.8, 0.9]
#threshold_list = [0.4]
#threshold_list = [1.0, 5.0 10.0, 15.0, 20.0]

start_time = time.time()

for thresh in threshold_list:

    #Print Threshold:
    print('\nCommenced for Similarity Threshold:', thresh)
    
    #X = DD_Data['clean_text'][0]
    #Y = list(DD_Data['ICD9_CODE'])
    
    predicted_codes_index = []
    actual_codes_index = []

    #m = 1#len(Y)

    for i in range(m):
        #text = X#[i]
        #print(i)
        #Convert Actual Codes to indices. Consider only those codes present in icd_codes
        act_icd_indices = [dict_code_index[y] for y in Y[i][2:-2].split("', '") if y in icd_codes] #To get rid of '' and ,

        #Initialize Prediction list. This will contain indices of similar icd codes
        pred = []

        #Apply thresholding to obtain indices of similar embeddings
        #print(sim)
        threshold_filter = sim[i] > thresh

        
        #Similarity >= threshold --> similarity 
        #Similarity < threshold --> 0
        sim_post_threshold = sim[i]*threshold_filter

        #Add indices of non zero elements to pred
        pred.extend(np.nonzero(sim_post_threshold)[0].tolist())

        #Consider the unique indices
        pred = set(pred)
        #print(pred,"\n",act_icd_indices)
        #Convert list of indices to hot vector
        pred_icd_vector = generate_binary(pred)
        act_icd_vector = generate_binary(act_icd_indices)

        #Append
        #print(act_icd_vector,"\n",pred_icd_vector)
        predicted_codes_index.append(pred_icd_vector)
        actual_codes_index.append(act_icd_vector)


        #if i%100 == 0 and i != 0:
        #    print(i, ' sentences done.')

    #Calculate Precision, Recall, F1 Score for this threshold
    precision, recall, f1_score, support = precision_recall_fscore_support(actual_codes_index, predicted_codes_index, average = 'micro')
    specificity = calc_specificity(predicted_codes_index, actual_codes_index)

    #Add to results Dataframe:
    results = results.append({'Number of Observations': m, 
                    'Number of ICD Codes': NUM_ICD_CODES, 
                    'Threshold': thresh,
                    'Precision': precision, 
                    'Recall': recall, 
                    'F1-Score': f1_score, 
                    'Specificity': specificity}, ignore_index=True)

    print('\nPrecision: ', precision, ' Recall: ', recall, ' F1-Score: ', f1_score, ' Specificity:', specificity)


time_elapsed = time.time() - start_time
print('\nTime elapsed: ', time_elapsed/60.0, ' minutes')    
    
#Print Results
RESULT_FILENAME = 'Results_Experiment_Results_RWMS_Negation_Documentwise_' + str(DD_Data.shape[0]) + '_records_' + str(NUM_ICD_CODES) + '_codes_' + '.xlsx'
#RESULT_FILENAME = 'Experiment_Results_NP_RWMS_Documentwise_' + str(DD_Data.shape[0]) + '_records_' + str(NUM_ICD_CODES) + '_codes_' + '.xlsx'
results.to_excel(RESULT_FILENAME) 
print('Results Saved!')
results